#Avaliar exclusão de dados de 2024

Se tiver dados desse mesmo período repetidos, preciso limpar, pois estarão duplicados, pois agora só preciso de dados de 2025.

In [0]:
df = spark.table("bronze.spotify.streaming_history_audio_2024_2025_10")
df = df.select(
    "master_metadata_album_album_name",
    "master_metadata_album_artist_name",
    "master_metadata_track_name",
    "ms_played",
    "offline_timestamp",
    "platform",
    "reason_end",
    "reason_start",
    "shuffle",
    "skipped",
    "spotify_track_uri",
    "ts"
)
display(df)

In [0]:
from pyspark.sql.functions import min, max

display(df.select(min("ts").alias("min_ts")))

In [0]:
display(df.select(max("ts").alias("max_ts")))

In [0]:
from pyspark.sql.functions import col

df_2024 = df.filter(col("ts").between("2024-01-01", "2024-12-31"))
silver_df = spark.table("silver.spotify_eng.tb_spotify_analise").filter(
    (col("ts") >= "2024-12-24T15:01:14.000+00:00") & (col("ts") <= "2024-12-31")
)

join_keys = [
    "ts",
    "master_metadata_album_album_name",
    "master_metadata_album_artist_name",
    "master_metadata_track_name",
    "ms_played",
    "platform",
    "reason_end",
    "reason_start",
    "shuffle",
    "skipped",
    "spotify_track_uri",
    "offline_timestamp"
]

left_exclusive = df_2024.join(silver_df, on=join_keys, how="left_anti")
right_exclusive = silver_df.join(df_2024, on=join_keys, how="left_anti")

left_exclusive_selected = left_exclusive.select("spotify_track_uri", "ts",  "master_metadata_album_album_name","master_metadata_album_artist_name", "master_metadata_track_name")
right_exclusive_selected = right_exclusive.select("spotify_track_uri", "ts",  "master_metadata_album_album_name","master_metadata_album_artist_name", "master_metadata_track_name")

full_outer_exclusive = left_exclusive_selected.union(right_exclusive_selected)

display(full_outer_exclusive)

# Validar união de 2025
Validar se posso fazer uma união sem medo das tabelas de 2025


In [0]:
df1_2025 = df.filter(col("ts") >= "2025-01-01")

df2_2025 = spark.table("bronze.spotify.streaming_history_audio_2025_11")

join_keys = [
    "ts",
    "master_metadata_album_album_name",
    "master_metadata_album_artist_name",
    "master_metadata_track_name",
    "ms_played",
    "platform",
    "reason_end",
    "reason_start",
    "shuffle",
    "skipped",
    "spotify_track_uri",
    "offline_timestamp",
]

inner_exclusive = df1_2025.join(df2_2025, on=join_keys, how="inner")
display(inner_exclusive)


In [0]:
display(df1_2025.select(min("ts").alias("min_ts"), max("ts").alias("max_ts")))
display(df2_2025.select(min("ts").alias("min_ts"), max("ts").alias("max_ts")))